<a href="https://colab.research.google.com/github/sarra-gt/data-analytics/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets torch scikit-learn


In [ ]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
dataset = load_dataset("imdb")
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_function(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",

    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


In [ ]:
trainer.train()


Step,Training Loss
500,0.394900
1000,0.322700
1500,0.316600
2000,0.292800
2500,0.286400
3000,0.281400
3500,0.210400
4000,0.188800
4500,0.164300
5000,0.170900


TrainOutput(global_step=6250, training_loss=0.2476645068359375, metrics={'train_runtime': 2244.5945, 'train_samples_per_second': 22.276, 'train_steps_per_second': 2.784, 'total_flos': 6577776384000000.0, 'train_loss': 0.2476645068359375, 'epoch': 2.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])

y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Precision: 0.919613128270176
Recall: 0.928
F1-score: 0.9237875288683602


In [ ]:
import torch

def predict_sentiment(text):
    model.eval()
    device = next(model.parameters()).device

    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )

    # move inputs to same device as model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()

    return prediction


In [ ]:
predict_sentiment("At first, the movie feels promising: the cinematography is polished and the opening scenes hint at something emotionally engaging. However, as the story progresses, that initial excitement slowly fades. The characters are underdeveloped, and despite a few strong performances, the dialogue often feels forced and repetitive. The film isn’t terrible — there are moments of genuine tension and even a couple of scenes that work surprisingly well — but it’s also far from good. By the final act, the pacing becomes exhausting, and what should have been a powerful conclusion ends up feeling drawn-out and strangely empty. I wouldn’t call it a complete waste of time, yet I can’t honestly say I’d recommend it unless you have very low expectations or are simply curious.")


0

In [ ]:
predict_sentiment("""
ok so like i dont even know where to start this movie was kinda ok i guess?? the beginning was ok but then like it got super boring real fast i was checking my phone 100 times the actors??? idk some were fine some like terrible the story made zero sense sometimes?? like wtf the ending?? dont even get me started i was like waiting for something big to happen but nah nothing it was just meh honestly i wouldnt watch again maybe if ur desperate for something to do or ur bored as hell but dont expect anything good lmao
""")


0